In [1]:
from bs4 import BeautifulSoup as bs
import csv
import requests as rq
import pandas as pd
import numpy as np
import re
import time

In [2]:
def headers(soup): 
    table_headers = soup.find('thead')
    table_headers = table_headers.find('tr').text
    time.sleep(1)
    table_headers = table_headers.strip()
    table_headers = table_headers.split('\n')
    table_headers.insert(0,'Year')
    table_headers = np.array(np.matrix(table_headers))
    # print(table_headers)
    return table_headers

def table_data(soup,headers):
    data = []
    name_data = []
    player_data = []

    body = soup.find('body')
    content_div = body.find('div',id = 'content')
    title = content_div.find('h1').text.strip()
    year = title.split()
    draft_year = [year[0]]
    
    time.sleep(1)
    table_data = soup.find('tbody')
    table_rows = table_data.find_all('tr')
    table_names = table_data.find_all('th')
    for names in table_names:
        name = names.text.strip()
        if name not in headers:
            name_data.append(name)
        player_names = np.array(name_data)
    attendees = len(name_data)

    draft_year = draft_year * attendees

    # takes all the table rows that have actual people data.
    table_rows = table_data.find_all('tr', class_= lambda x : x != 'thead')
    for row in table_rows:
        info = row.find_all('td')
        info = [element.text.strip() for element in info]
        # removes all the html waffle and allows you to .text() using the for loop for each element in info
        data.append([element for element in info])
        player_measureables = np.array(data)
    player_data = np.column_stack((draft_year, player_names,player_measureables))

    return player_data

url = 'https://www.pro-football-reference.com/draft/2023-combine.htm'
combine = bs(rq.get(url).content,'html.parser')
table_headers = headers(combine)
player_data = table_data(combine,table_headers)

# print(table_headers)
# print(player_data[15])


In [3]:
# Headers remain the same throughout the years of data.


url = 'https://www.pro-football-reference.com/draft/2023-combine.htm'
combine = bs(rq.get(url).content,'html.parser')
table_headers = headers(combine)


player_data = table_data(combine,table_headers)


print(len(player_data))
print(table_headers)
print(player_data[15])

# with open('2023_combine_data.csv','w') as f:
#     writer = csv.writer(f)
#     writer.writerows(combine_data)

319
[['Year' 'Player' 'Pos' 'School' 'College' 'Ht' 'Wt' '40yd' 'Vertical'
  'Bench' 'Broad Jump' '3Cone' 'Shuttle' 'Drafted (tm/rnd/yr)']]
['2023' 'Habakkuk Baldonado' 'EDGE' 'Pittsburgh' 'College Stats' '6-4'
 '251' '4.78' '35.0' '21' '120' '7.11' '4.44' '']


In [14]:
urls = []

for year in range(2000,2024+1):
    urls.append(f'https://www.pro-football-reference.com/draft/{year}-combine.htm')
# ^ creating list of URLs, for some reason needs to be 1 longer than makes sense :/

all_prospects = np.empty((0,14),dtype=str)
for url in urls:
    combine = bs(rq.get(url).content,'html.parser')
    table_headers = headers(combine)
    player_data = table_data(combine,table_headers)
    time.sleep(1)
    # ^ put the delay in to avoid overwhelming server, idk if thats a thing but was a potential fix online and it worked.
    all_prospects = np.append(all_prospects, player_data, axis=0)
    print('This years draft has',np.shape(player_data)[0],'prospects.')
    print('Totalling to',np.shape(all_prospects),'so far.')
# ^ runs functions for each year


with open('combine_data.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows(table_headers)
    writer.writerows(all_prospects)
# ^ writes to csv file so I dont have to run it all each time.

This years draft has 321 prospects.
Totalling to (321, 14) so far.
This years draft has 321 prospects.
Totalling to (642, 14) so far.
This years draft has 320 prospects.
Totalling to (962, 14) so far.
This years draft has 319 prospects.
Totalling to (1281, 14) so far.
This years draft has 335 prospects.
Totalling to (1616, 14) so far.
This years draft has 331 prospects.
Totalling to (1947, 14) so far.
This years draft has 328 prospects.
Totalling to (2275, 14) so far.
This years draft has 326 prospects.
Totalling to (2601, 14) so far.
This years draft has 330 prospects.
Totalling to (2931, 14) so far.
This years draft has 327 prospects.
Totalling to (3258, 14) so far.
This years draft has 326 prospects.
Totalling to (3584, 14) so far.
This years draft has 329 prospects.
Totalling to (3913, 14) so far.
This years draft has 324 prospects.
Totalling to (4237, 14) so far.
This years draft has 332 prospects.
Totalling to (4569, 14) so far.
This years draft has 333 prospects.
Totalling to (4

In [15]:
df = pd.read_csv('combine_data.csv', sep=',',encoding='utf-8',header=0)
df.head()
# have to rerun this regularly.

,Year,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
0,2000,John Abraham,OLB,South Carolina,NaN,6-4,252.0,4.55,NaN,NaN,NaN,NaN,NaN,New York Jets / 1st / 13th pick / 2000
1,2000,Shaun Alexander,RB,Alabama,College Stats,6-0,218.0,4.58,NaN,NaN,NaN,NaN,NaN,Seattle Seahawks / 1st / 19th pick / 2000
2,2000,Darnell Alford,OT,Boston Col.,NaN,6-4,334.0,5.56,25.0,23.0,94.0,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000
3,2000,Kyle Allamon,TE,Texas Tech,NaN,6-2,253.0,4.97,29.0,NaN,104.0,7.29,4.49,NaN
4,2000,Rashard Anderson,CB,Jackson State,NaN,6-2,206.0,4.55,34.0,NaN,123.0,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000


In [16]:
df = df.drop(columns='College')
df = df.rename(columns={'School' : 'College',
                   'Ht' : 'Height',
                   'Wt' : 'Weight'})
# df.head()
df.sort_values('Height', ascending = False)
# ^ ascending = False is descending

,Year,Player,Pos,College,Height,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
1801,2005,Samuel Lightbody,OT,Michigan State,6-9,321.0,5.62,30.0,NaN,108.0,7.95,4.41,NaN
6086,2018,Kolton Miller,OT,UCLA,6-9,309.0,4.95,31.5,24.0,121.0,7.34,4.49,Oakland Raiders / 1st / 15th pick / 2018
8004,2024,Joe Alt,OT,Notre Dame,6-9,321.0,5.05,28.0,NaN,112.0,7.31,4.51,NaN
1012,2003,Brennan Curtin,OT,Notre Dame,6-9,318.0,5.28,32.0,NaN,100.0,7.98,4.87,Green Bay Packers / 6th / 212th pick / 2003
6080,2018,Mike McGlinchey,OT,Notre Dame,6-8,309.0,NaN,28.5,24.0,105.0,NaN,NaN,San Francisco 49ers / 1st / 9th pick / 2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7265,2021,Elerson Smith,DL,Northern Iowa,NaN,NaN,4.71,41.5,26.0,127.0,7.00,4.39,New York Giants / 4th / 116th pick / 2021
7300,2021,Khyiris Tonga,DL,BYU,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,Chicago Bears / 7th / 250th pick / 2021
7306,2021,Carl Tucker,TE,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7321,2021,Garret Wallow,LB,TCU,NaN,NaN,4.65,32.5,22.0,122.0,6.87,4.12,Houston Texans / 5th / 170th pick / 2021


In [17]:
df[['Team', 'Round', 'Pick','Year2']] = df['Drafted (tm/rnd/yr)'].str.split(' / ', expand=True)
df = df.drop(columns={'Year2', 'Drafted (tm/rnd/yr)'})
df['Team'].fillna('Undrafted', inplace=True)
df['Round'].fillna('Undrafted', inplace=True)
df['Pick'].fillna('Undrafted', inplace=True)
# ^ inplace = True keeps rest of columns
df[['Feet','Inches']] = df['Height'].str.split('-', expand=True)
df['Height'] = df['Feet'] + "'" + df['Inches'] + "\""
df = df.drop(columns={'Feet', 'Inches'})
df.head()

,Year,Player,Pos,College,Height,Weight,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Team,Round,Pick
0,2000,John Abraham,OLB,South Carolina,"6'4""",252.0,4.55,NaN,NaN,NaN,NaN,NaN,New York Jets,1st,13th pick
1,2000,Shaun Alexander,RB,Alabama,"6'0""",218.0,4.58,NaN,NaN,NaN,NaN,NaN,Seattle Seahawks,1st,19th pick
2,2000,Darnell Alford,OT,Boston Col.,"6'4""",334.0,5.56,25.0,23.0,94.0,8.48,4.98,Kansas City Chiefs,6th,188th pick
3,2000,Kyle Allamon,TE,Texas Tech,"6'2""",253.0,4.97,29.0,NaN,104.0,7.29,4.49,Undrafted,Undrafted,Undrafted
4,2000,Rashard Anderson,CB,Jackson State,"6'2""",206.0,4.55,34.0,NaN,123.0,7.18,4.15,Carolina Panthers,1st,23rd pick


In [18]:
df = df[['Year', 'Player', 'Pos', 'College', 'Height', 'Weight', '40yd' ,'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Team', 'Round', 'Pick']]
df.to_csv('clean_combine_data.csv', index=True)




Rerun box below




In [30]:
df = pd.read_csv('clean_combine_data.csv', sep=',',encoding='utf-8',header=0)
df = df.rename(columns ={'Unnamed: 0': 'Index'})
# df.head()
# have to rerun this regularly.

In [31]:
import scipy.stats as stats
tests = {'Height': 1,
         'Weight': 1,
         '40yd': 0,
         'Vertical': 1,
         'Bench': 1,
         'Broad Jump': 1,
         '3Cone': 0,
         'Shuttle': 0}
# 1 if higher is better, 0 if lower is better. For timed events (40yd, 3Cone, Shuttle), a better result is lower -> higher %ile.
for col, val in tests.items():
    df_test = df.dropna(subset=[col])
    df_test = df_test.sort_values(col, ascending = True)
    df_test[col+'_%ile'] = stats.percentileofscore(df_test[col],df_test[col])
    if val == 0:
        df_test[col+'_%ile'] = 100 - df_test[col+'_%ile']
    # if statements flips %iles based on dict above
    df_test = df_test[['Index',col+'_%ile']]
    # df_test = df_test.replace('NaN','0')
    df = df.merge(df_test, how='outer', on='Index')
# Finds percentiles for each test and add col w that value. NaN if player didnt compete.
df.fillna(0,inplace=True)

In [32]:
def Pick_N(df_avrg, n):
    df_avrg = df[['40yd_%ile','Vertical_%ile', 'Bench_%ile', 'Broad Jump_%ile', '3Cone_%ile', 'Shuttle_%ile']]
    # Create a new column to store the average of best n results
    # df_avrg = df_avrg.replace('NaN', '0')
    df_avrg[f'Average_of_{n}'] = df_avrg.apply(lambda row: row.nlargest(n).mean(), axis=1)
    df[f'Average_of_{n}'] = df_avrg[f'Average_of_{n}']
    return df

df_6 = Pick_N(df,6)
# df_complete['Average_ofN'] = df.assign(df_complete)
df_6.tail()

/var/folders/2q/w6cfp68d293ccyw_4ysbw26c0000gn/T/ipykernel_28140/675849103.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Index,Year,Player,Pos,College,Height,Weight,40yd,Vertical,Bench,...,Pick,Height_%ile,Weight_%ile,40yd_%ile,Vertical_%ile,Bench_%ile,Broad Jump_%ile,3Cone_%ile,Shuttle_%ile,Average_of_6
8315,8315,2024,Roman Wilson,WR,Michigan,"5'11""",185.0,4.39,0.0,12.0,...,Undrafted,10.535520,4.622710,95.606338,0.000000,8.075536,0.000000,0.0,0.0,17.280312
8316,8316,2024,Mekhi Wingo,DT,LSU,"6'0""",284.0,4.85,31.5,25.0,...,Undrafted,26.335786,75.711186,33.937926,34.949937,75.593092,25.087191,0.0,0.0,28.261358
8317,8317,2024,Xavier Worthy,WR,Texas,"5'11""",165.0,4.21,41.0,0.0,...,Undrafted,10.535520,0.150675,99.986904,97.794118,0.000000,97.154407,0.0,0.0,49.155905
8318,8318,2024,Jaylen Wright,RB,Tennessee,"5'11""",210.0,4.38,38.0,0.0,...,Undrafted,10.535520,30.876326,96.444474,88.767209,0.000000,99.064680,0.0,0.0,47.379394
8319,8319,2024,Zak Zinter,G,Michigan,"6'6""",309.0,0.00,0.0,0.0,...,Undrafted,95.181522,87.662729,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [33]:
df_best = df.sort_values(df.columns[-1], ascending = False)
df_best.head()

,Index,Year,Player,Pos,College,Height,Weight,40yd,Vertical,Bench,...,Pick,Height_%ile,Weight_%ile,40yd_%ile,Vertical_%ile,Bench_%ile,Broad Jump_%ile,3Cone_%ile,Shuttle_%ile,Average_of_6
4778,4778,2014,Jerick McKinnon,RB,Georgia Southern,"5'9""",209.0,4.41,40.5,32.0,...,96th pick,18.827644,29.622710,93.877685,96.831977,95.501993,97.899493,89.591754,86.963794,93.444449
7348,7348,2021,Marco Wilson,CB,Florida,"5'11""",191.0,4.37,43.5,26.0,...,136th pick,10.535520,9.582932,97.164746,99.734043,80.176504,99.635384,91.956346,85.615899,92.380487
168,168,2000,Curtis Keaton,RB,West Virginia,"5'10""",214.0,4.41,41.5,23.0,...,97th pick,3.063563,35.547252,93.877685,98.576345,64.537863,97.154407,90.268795,98.720976,90.522679
2003,2003,2006,Vernon Davis,TE,Maryland,"6'3""",254.0,4.38,42.0,33.0,...,6th pick,64.232300,66.092093,96.444474,99.131727,96.498387,93.746037,72.402991,80.293192,89.752801
4432,4432,2013,Christine Michael,RB,Texas A&M,"5'10""",220.0,4.54,43.0,27.0,...,62nd pick,3.063563,41.037850,73.801729,99.601064,84.190548,87.666455,96.796686,95.749705,89.634364


In [36]:
import plotly.express as px
df_radar = df_best.head(1)
df_radar
radar_cols = df_radar.columns[18:24]
radar_vals = df_radar[radar_cols].values.tolist()
radar_vals = radar_vals[0]

print(radar_cols)
print(radar_vals)
print(df_radar['Player'])
fig = px.line_polar(df_radar,r = radar_vals,theta= radar_cols, line_close=True)
fig.update_traces(fill='toself')
fig.show()

Index(['40yd_%ile', 'Vertical_%ile', 'Bench_%ile', 'Broad Jump_%ile',
       '3Cone_%ile', 'Shuttle_%ile'],
      dtype='object')
[93.87768465165007, 96.8319774718398, 95.50199278800531, 97.89949270767279, 89.59175424413905, 86.9637937819756]
4778    Jerick McKinnon
Name: Player, dtype: object
